#### Определим переменные путей

In [1]:
%env MAIN = /Users/vladimir/temp/AnacondaProjects/numbers/numbers_recognition
%env TFP = /Users/vladimir/tensorflow
%env ANDROID = /Users/vladimir/temp/androidYolo
%env PB_MAIN_DETECT = /Users/vladimir/temp/AnacondaProjects/numbers_recognition/detection/tf-exports
%env PB_MAIN_TEXT_DETECT = /Users/vladimir/temp/AnacondaProjects/numbers_recognition/text-detection-ctpn/tf-exports
%env PB_MAIN_REC = /Users/vladimir/temp/AnacondaProjects/numbers_recognition/recognition/tf-exports
%env PB_MAIN_TEXT_REC = /Users/vladimir/temp/AnacondaProjects/numbers_recognition/text-recognition/tf-exports

env: MAIN=/Users/vladimir/temp/AnacondaProjects/numbers/numbers_recognition
env: TFP=/Users/vladimir/tensorflow
env: ANDROID=/Users/vladimir/temp/androidYolo
env: PB_MAIN_DETECT=/Users/vladimir/temp/AnacondaProjects/numbers_recognition/detection/tf-exports
env: PB_MAIN_TEXT_DETECT=/Users/vladimir/temp/AnacondaProjects/numbers_recognition/text-detection-ctpn/tf-exports
env: PB_MAIN_REC=/Users/vladimir/temp/AnacondaProjects/numbers_recognition/recognition/tf-exports
env: PB_MAIN_TEXT_REC=/Users/vladimir/temp/AnacondaProjects/numbers_recognition/text-recognition/tf-exports


In [37]:
%env MAIN = /Users/rimidalv/AnacondaProjects/numbers_recognition
%env TFP = /Users/rimidalv/tensorflow
%env ANDROID = /Users/rimidalv/AndroidStudioProjects/agronumbers
%env PB_MAIN_DETECT = /Users/rimidalv/AnacondaProjects/numbers_recognition/detection/tf-exports
%env PB_MAIN_TEXT_DETECT = /Users/rimidalv/AnacondaProjects/numbers_recognition/text-detection-ctpn/tf-exports
%env PB_MAIN_REC = /Users/rimidalv/AnacondaProjects/numbers_recognition/recognition/tf-exports

env: MAIN=/Users/rimidalv/AnacondaProjects/numbers_recognition
env: TFP=/Users/rimidalv/tensorflow
env: ANDROID=/Users/rimidalv/AndroidStudioProjects/agronumbers
env: PB_MAIN_DETECT=/Users/rimidalv/AnacondaProjects/numbers_recognition/detection/tf-exports
env: PB_MAIN_TEXT_DETECT=/Users/rimidalv/AnacondaProjects/numbers_recognition/text-detection-ctpn/tf-exports
env: PB_MAIN_REC=/Users/rimidalv/AnacondaProjects/numbers_recognition/recognition/tf-exports


# RECOGNITION

##### Подготовим модель для выгрузки в проект

In [16]:
!cd $TFP && bazel build tensorflow/python/tools:freeze_graph
!$TFP/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=$PB_MAIN_REC/protobuf_path.pbtxt \
--input_checkpoint=$PB_MAIN_REC/checkpoint_path.ckpt \
--output_graph=$PB_MAIN_REC/frozen_graph.pb \
--output_node_names=softmax/truediv 

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:freeze_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/python/tools:freeze_graph up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/python/tools/freeze_graph
[1 / 1] no action
INFO: Elapsed time: 0.486s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action
/anaconda3/lib/python3.6/site-pac

 quantize_nodes round_weights

In [17]:
!cd $TFP && bazel build tensorflow/tools/graph_transforms:transform_graph
!$TFP/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=$PB_MAIN_REC/frozen_graph.pb \
--out_graph=$PB_MAIN_REC/frozen_opt_rec.pb \
--inputs='the_input:0' \
--outputs='softmax/truediv:0' \
--transforms='add_default_attributes remove_attribute(attribute_name=_class) remove_nodes(op=CheckNumerics) fold_constants(ignore_errors=true) fold_batch_norms fold_old_batch_norms strip_unused_nodes sort_by_execution_order'

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
INFO: Analysed target //tensorflow/tools/graph_transforms:transform_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/tools/graph_transforms:transform_graph up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/tools/graph_transforms/transform_graph
[1 / 1] no action
INFO: Elapsed time: 0.447s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action
2018-03-09 14:47:38.467447: I tensorflow/tools/graph_transforms/transform_graph.cc:264] Applying add_default_attributes
2018-03-09 14:47:38.495720: I tensorflow/tools/graph_transfo

In [18]:
!cp $PB_MAIN_REC/frozen_opt_rec.pb $ANDROID/assets

##### Скачаем и скомпилим утилиту, которая сгенерирует файл ops_to_register.h для регистрации всех Node в Tensorflow взятых из нашей модели (если точнее из profobuf файла)

In [19]:
!cd $TFP && bazel build tensorflow/python/tools:print_selective_registration_header

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:print_selective_registration_header (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/python/tools:print_selective_registration_header up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow

##### Сгенерируем сам файл

In [20]:
!$TFP/bazel-bin/tensorflow/python/tools/print_selective_registration_header \
--graphs=$PB_MAIN_REC/frozen_opt_rec.pb > $ANDROID/etc/ops_to_register_rec.h

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
/Users/vladimir/temp/AnacondaProjects/numbers_recognition/text-recognition/tf-exports

SyntaxError: invalid syntax (<ipython-input-4-b15e70d26b48>, line 1)

In [3]:
!ls $PB_MAIN_TEXT_REC

ls: /Users/vladimir/AnacondaProjects/numbers_recognition/text-recognition/tf-exports: No such file or directory


# TEXT RECOGNITION

##### Подготовим модель для выгрузки в проект

In [5]:
!cd $TFP && bazel build tensorflow/python/tools:freeze_graph
!$TFP/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=$PB_MAIN_TEXT_REC/protobuf_path.pbtxt \
--input_checkpoint=$PB_MAIN_TEXT_REC/checkpoint_path.ckpt \
--output_graph=$PB_MAIN_TEXT_REC/frozen_graph.pb \
--output_node_names=softmax/truediv 

................
Loading: 
Loading: 0 packages loaded
Loading: 0 packages loaded
    currently loading: tensorflow/python/tools
Analyzing: target //tensorflow/python/tools:freeze_graph (1 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (2 packages loaded)
    currently loading: @local_config_cc//
Analyzing: target //tensorflow/python/tools:freeze_graph (3 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (7 packages loaded)
    currently loading: tensorflow/core ... (3 packages)
Analyzing: target //tensorflow/python/tools:freeze_graph (9 packages loaded)
    currently loading: tensorflow/core ... (2 packages)
Analyzing: target //tensorflow/python/tools:freeze_graph (13 packages loaded)
    currently loading: tensorflow/core ... (2 packages)
Analyzing: target //tensorflow/python/tools:freeze_graph (15 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (15 packages loaded)
Analyzing: target //tensorflow/python/tools

Building: no action
[0 / 4] [-----] BazelWorkspaceStatusAction stable-status.txt
[1 / 5] no action
Target //tensorflow/python/tools:freeze_graph up-to-date:
[4,877 / 4,877] no action
  bazel-bin/tensorflow/python/tools/freeze_graph
[4,877 / 4,877] no action
INFO: Elapsed time: 139,188s, Critical Path: 6,25s
[4,877 / 4,877] no action
INFO: Build completed successfully, 1 total action
/Users/vladimir/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
Use the retry module or similar alternatives.
2018-03-16 10:38:17.564332: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA
Converted 20 variables to const ops.


 quantize_nodes round_weights

In [6]:
!cd $TFP && bazel build tensorflow/tools/graph_transforms:transform_graph
!$TFP/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=$PB_MAIN_TEXT_REC/frozen_graph.pb \
--out_graph=$PB_MAIN_TEXT_REC/frozen_opt_text_rec.pb \
--inputs='the_input:0' \
--outputs='softmax/truediv:0' \
--transforms='add_default_attributes remove_attribute(attribute_name=_class) remove_nodes(op=CheckNumerics) fold_constants(ignore_errors=true) fold_batch_norms fold_old_batch_norms strip_unused_nodes sort_by_execution_order'

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
INFO: Analysed target //tensorflow/tools/graph_transforms:transform_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 7] no action
[4 / 7] no action
[6 / 7] no action
Target //tensorflow/tools/graph_transforms:transform_graph up-to-date:
[7 / 7] no action
  bazel-bin/tensorflow/tools/graph_transforms/transform_graph
[7 / 7] no action
INFO: Elapsed time: 3,945s, Critical Path: 3,22s
[7 / 7] no action
INFO: Build completed successfully, 1 total action
2018-03-16 10:38:23.281580: I tensorflow/tools/graph_transforms/transform_graph.cc:264] Applying add_default_attributes
2018-03-16 10:38:23.298249: I tensorflow/tools/graph_transforms/tra

In [7]:
!cp $PB_MAIN_TEXT_REC/frozen_opt_text_rec.pb $ANDROID/assets

##### Скачаем и скомпилим утилиту, которая сгенерирует файл ops_to_register.h для регистрации всех Node в Tensorflow взятых из нашей модели (если точнее из profobuf файла)

In [19]:
!cd $TFP && bazel build tensorflow/python/tools:print_selective_registration_header

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:print_selective_registration_header (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/python/tools:print_selective_registration_header up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow

##### Сгенерируем сам файл

In [20]:
!$TFP/bazel-bin/tensorflow/python/tools/print_selective_registration_header \
--graphs=$PB_MAIN_TEXT_REC/frozen_opt_rec.pb > $ANDROID/etc/ops_to_register_rec.h

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# DETECTION

##### Подготовим модель для выгрузки в проект

quantize_nodes round_weights

In [21]:
!cd $TFP && bazel build tensorflow/python/tools:freeze_graph
!$TFP/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=$PB_MAIN_DETECT/yolo-tiny-v1.pbtxt \
--input_checkpoint=$PB_MAIN_DETECT/yolo-tiny-v1.ckpt \
--output_graph=$PB_MAIN_DETECT/frozen_graph.pb \
--output_node_names=final_conv2d_layer/BiasAdd

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:freeze_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/python/tools:freeze_graph up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/python/tools/freeze_graph
[1 / 1] no action
INFO: Elapsed time: 0.418s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action
/anaconda3/lib/python3.6/site-pac

In [22]:
!cd $TFP && bazel build tensorflow/tools/graph_transforms:transform_graph
!$TFP/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=$PB_MAIN_DETECT/frozen_graph.pb \
--out_graph=$PB_MAIN_DETECT/frozen_opt_detect.pb \
--inputs='input_1:0' \
--outputs='final_conv2d_layer/BiasAdd:0' \
--transforms='add_default_attributes remove_attribute(attribute_name=_class) remove_nodes(op=CheckNumerics) fold_constants(ignore_errors=true) fold_batch_norms fold_old_batch_norms round_weights strip_unused_nodes sort_by_execution_order'

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
INFO: Analysed target //tensorflow/tools/graph_transforms:transform_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/tools/graph_transforms:transform_graph up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/tools/graph_transforms/transform_graph
[1 / 1] no action
INFO: Elapsed time: 0.469s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action
2018-03-09 14:48:10.115025: I tensorflow/tools/graph_transforms/transform_graph.cc:264] Applying add_default_attributes
2018-03-09 14:48:10.178211: I tensorflow/tools/graph_transfo

In [23]:
!cp $PB_MAIN_DETECT/frozen_opt_detect.pb $ANDROID/assets

##### Скачаем и скомпилим утилиту, которая сгенерирует файл ops_to_register.h для регистрации всех Node в Tensorflow взятых из нашей модели (если точнее из profobuf файла)

In [24]:
!cd $TFP && bazel build tensorflow/python/tools:print_selective_registration_header

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:print_selective_registration_header (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/python/tools:print_selective_registration_header up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow

##### Сгенерируем сам файл

In [25]:
!$TFP/bazel-bin/tensorflow/python/tools/print_selective_registration_header \
--graphs=$PB_MAIN_DETECT/frozen_opt_detect.pb > $ANDROID/etc/ops_to_register_detect.h

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# TEXT DETECTION

##### Подготовим модель для выгрузки в проект

In [39]:
!cd $TFP && bazel build tensorflow/python/tools:freeze_graph
!$TFP/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=$PB_MAIN_TEXT_DETECT/v1.pbtxt \
--input_checkpoint=$PB_MAIN_TEXT_DETECT/v1.ckpt \
--output_graph=$PB_MAIN_TEXT_DETECT/frozen_graph.pb \
--output_node_names=rois/Reshape_4

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:freeze_graph (1 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (11 packages loaded)
    currently loading: tensorflow/python/eager
Analyzing: target //tensorflow/python/tools:freeze_graph (11 packages loaded)
    currently loading: tensorflow/python/eager ... (2 packages)
Analyzing: target //tensorflow/python/tools:freeze_graph (26 packages loaded)
    currently loading: @cython// ... (8 packages)
Analyzing: target //tensorflow/python/tools:freeze_graph (41 packages loaded)
    currently loading: tensorflow/core/profiler/internal/advisor
Analyzing: target //tensorflow/python/tools:freeze_graph (124 packages loaded)
    currently loading: tensorflow/compiler/jit
Analyzing: target //tensorflow/python/tools:freeze_graph (164 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (165 packages loaded)
Analyzing: target //tensorflow/python/tools:freeze_graph (165 pa

 quantize_nodes round_weights

In [40]:
!cd $TFP && bazel build tensorflow/tools/graph_transforms:transform_graph
!$TFP/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=$PB_MAIN_TEXT_DETECT/frozen_graph.pb \
--out_graph=$PB_MAIN_TEXT_DETECT/frozen_opt_text_detect.pb \
--inputs='Placeholder:0,Placeholder_1:0' \
--outputs='rois/Reshape_4:0' \
--transforms='add_default_attributes remove_attribute(attribute_name=_class) remove_nodes(op=CheckNumerics) fold_constants(ignore_errors=true) fold_batch_norms fold_old_batch_norms strip_unused_nodes sort_by_execution_order'

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
Analyzing: target //tensorflow/tools/graph_transforms:transform_graph (0 packa\
ges loaded)
INFO: Analysed target //tensorflow/tools/graph_transforms:transform_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/tools/graph_transforms:transform_graph up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/tools/graph_transforms/transform_graph
[1 / 1] no action
INFO: Elapsed time: 0.470s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action
2018-03-10 12:16:21.104645: I tensorflow/tools/graph_transforms/transform_graph.cc:264] Applying add_default_attributes
2018-03-10 12:16:21.202103: I tensorflow/tools/graph_transfo

In [41]:
!cp $PB_MAIN_TEXT_DETECT/frozen_opt_text_detect.pb $ANDROID/assets

##### Скачаем и скомпилим утилиту, которая сгенерирует файл ops_to_register.h для регистрации всех Node в Tensorflow взятых из нашей модели (если точнее из profobuf файла)

In [42]:
!cd $TFP && bazel build tensorflow/python/tools:print_selective_registration_header

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
Analyzing: target //tensorflow/python/tools:print_selective_registration_heade\
r (0 packages loaded)
INFO: Analysed target //tensorflow/python/tools:print_selective_registration_header (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 2] no action
Target //tensorflow/python/tools:print_selective_registration_header up-to-date:
[4 / 4] no action
  bazel-bin/tensorflow/python/tools/print_selective_registration_

##### Сгенерируем сам файл

In [43]:
!$TFP/bazel-bin/tensorflow/python/tools/print_selective_registration_header \
--graphs=$PB_MAIN_TEXT_DETECT/frozen_opt_text_detect.pb > $ANDROID/etc/ops_to_register_text_detect.h

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [36]:
!cd $TFP && bazel build tensorflow/tools/graph_transforms:summarize_graph && \
$TFP/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
--in_graph=$PB_MAIN_TEXT_DETECT/frozen_graph.pb

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/tools/graph_transforms:summarize_graph (0 packa\
ges loaded)
INFO: Analysed target //tensorflow/tools/graph_transforms:summarize_graph (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 17] [-----] //tensorflow/tools/graph_transforms:summarize_graph
[820 / 823] Compiling .../graph_transforms/summarize_graph_main.cc; 0s local
[820 / 823] Compiling .../graph_transforms/summarize_graph_main.cc; 1s local
[820 / 823] Compiling .../graph_transforms/summarize_graph_main.cc; 2s local
[820 / 823] Compiling .../graph_transforms/summarize_graph_main.cc; 3s local
[821 / 823] [-----] Linking .../graph_transforms/libsummarize_graph_main_lib.a
INFO: From Linking tensorflow/tools/graph_transforms/summarize_graph:
[822 / 823] Linking .../tools/graph_transforms/summarize_graph; 0s local
clang: warning: argument unused during compilation: '-pthread' [-Wunused-command-line-argument]
[822 / 823] Linki

In [26]:
!code $ANDROID/etc/ops_to_register_detect.h

In [27]:
!code $ANDROID/etc/ops_to_register_rec.h

In [53]:
!code $ANDROID/etc/ops_to_register_text_detect.h

In [54]:
!code $ANDROID/etc/ops_to_register.h

## Нужно объединить два предыдущих файла в один

In [55]:
!cp $ANDROID/etc/ops_to_register.h $TFP/tensorflow/core/framework/ops_to_register.h

##### (Опционально) Откроем и посмотрим что там нагенерировалось

!code $TFP/tensorflow/core/framework/ops_to_register.h

##### На основе файла ops_to_register.h нужно сгенерировать либу libtensorflow_inference.so, что бы положить его в Андроид проект (--copt="-D__ANDROID_TYPES_FULL__" \)

In [56]:
!cd $TFP && bazel build -c opt --copt="-DSELECTIVE_REGISTRATION" \
--copt="-DSUPPORT_SELECTIVE_REGISTRATION" \
//tensorflow/contrib/android:libtensorflow_inference.so \
--host_crosstool_top=@bazel_tools//tools/cpp:toolchain \
--crosstool_top=//external:android/crosstool --cpu=armeabi-v7a 

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Analyzing: target //tensorflow/contrib/android:libtensorflow_inference.so (0 p\
ackages loaded)
Ana

INFO: Analysed target //tensorflow/contrib/android:libtensorflow_inference.so (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[2 / 9] [-----] BazelWorkspaceStatusAction stable-status.txt
[229 / 242] 8 actions, 7 running
    Compiling tensorflow/c/c_api.cc; 0s local
    Compiling tensorflow/core/kernels/softplus_op.cc; 0s local
    Compiling tensorflow/core/kernels/strided_slice_op.cc; 0s local
    Compiling tensorflow/c/c_api_function.cc; 0s local
    Compiling tensorflow/core/kernels/dequantize_op.cc; 0s local
    Compiling tensorflow/core/kernels/function_ops.cc; 0s local
    Compiling tensorflow/core/kernels/lookup_table_init_op.cc; 0s local
    [Analy] Compiling tensorflow/core/kernels/split_op.cc
[229 / 242] 8 actions running
    Compiling tensorflow/c/c_api.cc; 0s local
    Compiling tensorflow/core/kernels/softplus_op.cc; 0s local
    Compiling tensorflow/core/kernels/strided_slice_op.cc; 0s local
    Compiling tensorflow/c/c_api_function.cc;

[233 / 246] 7 actions running
    Compiling tensorflow/c/c_api.cc; 10s local
    Compiling tensorflow/core/kernels/softplus_op.cc; 10s local
    Compiling tensorflow/core/kernels/strided_slice_op.cc; 10s local
    Compiling tensorflow/core/kernels/split_op.cc; 10s local
    Compiling tensorflow/core/kernels/pack_op.cc; 4s local
    Compiling tensorflow/core/kernels/one_hot_op.cc; 3s local
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 3s local
[233 / 246] 8 actions running
    Compiling tensorflow/c/c_api.cc; 10s local
    Compiling tensorflow/core/kernels/softplus_op.cc; 10s local
    Compiling tensorflow/core/kernels/strided_slice_op.cc; 10s local
    Compiling tensorflow/core/kernels/split_op.cc; 10s local
    Compiling tensorflow/core/kernels/pack_op.cc; 5s local
    Compiling tensorflow/core/kernels/one_hot_op.cc; 3s local
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 3s local
    Compiling tensorflow/core/kernels/cwise_op_left_shift.cc; 0s local
[233 / 24

[241 / 254] 8 actions running
    Compiling tensorflow/core/kernels/one_hot_op.cc; 12s local
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 12s local
    Compiling tensorflow/core/kernels/cwise_op_left_shift.cc; 9s local
    Compiling tensorflow/core/kernels/cwise_op_abs.cc; 4s local
    Compiling tensorflow/core/kernels/maxpooling_op.cc; 4s local
    Compiling .../core/kernels/quantized_resize_bilinear_op.cc; 1s local
    Compiling tensorflow/core/kernels/deep_conv2d.cc; 0s local
    Compiling tensorflow/core/kernels/gather_nd_op_cpu_impl_6.cc; 0s local
[241 / 254] 8 actions running
    Compiling tensorflow/core/kernels/one_hot_op.cc; 13s local
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 13s local
    Compiling tensorflow/core/kernels/cwise_op_left_shift.cc; 9s local
    Compiling tensorflow/core/kernels/cwise_op_abs.cc; 5s local
    Compiling tensorflow/core/kernels/maxpooling_op.cc; 4s local
    Compiling .../core/kernels/quantized_resize_bilinear_op.cc; 1

[245 / 258] 7 actions running
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 21s local
    Compiling tensorflow/core/kernels/cwise_op_left_shift.cc; 18s local
    Compiling tensorflow/core/kernels/maxpooling_op.cc; 12s local
    Compiling tensorflow/core/kernels/gather_nd_op_cpu_impl_6.cc; 8s local
    Compiling tensorflow/core/kernels/mirror_pad_op.cc; 6s local
    Compiling tensorflow/core/kernels/fifo_queue.cc; 3s local
    Compiling tensorflow/core/kernels/cwise_op_isnan.cc; 2s local
[245 / 258] 8 actions running
    Compiling tensorflow/core/kernels/cwise_op_greater.cc; 21s local
    Compiling tensorflow/core/kernels/cwise_op_left_shift.cc; 18s local
    Compiling tensorflow/core/kernels/maxpooling_op.cc; 13s local
    Compiling tensorflow/core/kernels/gather_nd_op_cpu_impl_6.cc; 8s local
    Compiling tensorflow/core/kernels/mirror_pad_op.cc; 6s local
    Compiling tensorflow/core/kernels/fifo_queue.cc; 3s local
    Compiling tensorflow/core/kernels/cwise_op_isnan.cc;

...

[725 / 736] 8 actions running
    Compiling tensorflow/core/common_runtime/direct_session.cc; 13s local
    Compiling tensorflow/core/common_runtime/device_mgr.cc; 6s local
    Compiling tensorflow/core/graph/graph_partition.cc; 5s local
    Compiling tensorflow/core/framework/function.cc; 4s local
    Compiling tensorflow/core/kernels/pooling_ops_common.cc; 4s local
    Compiling tensorflow/core/common_runtime/device.cc; 2s local
    Compiling tensorflow/core/common_runtime/placer.cc; 1s local
    Compiling tensorflow/core/util/equal_graph_def.cc; 1s local
[726 / 737] 7 actions running
    Compiling tensorflow/core/common_runtime/direct_session.cc; 14s local
    Compiling tensorflow/core/graph/graph_partition.cc; 5s local
    Compiling tensorflow/core/framework/function.cc; 5s local
    Compiling tensorflow/core/kernels/pooling_ops_common.cc; 4s local
    Compiling tensorflow/core/common_runtime/device.cc; 2s local
    Compiling tensorflow/core/common_runtime/placer.cc; 1s local
    C

[762 / 773] 7 actions running
    Compiling tensorflow/core/framework/node_def_util.cc; 7s local
    //tensorflow/core:android_tensorflow_lib_lite; 6s local
    Compiling .../core/common_runtime/threadpool_device_factory.cc; 4s local
    Compiling .../core/common_runtime/optimization_registry.cc; 3s local
    Compiling tensorflow/core/common_runtime/local_device.cc; 2s local
    Compiling tensorflow/core/graph/quantize_training.cc; 1s local
    Compiling tensorflow/core/common_runtime/threadpool_device.cc; 0s local
[762 / 773] 8 actions running
    Compiling tensorflow/core/framework/node_def_util.cc; 8s local
    //tensorflow/core:android_tensorflow_lib_lite; 6s local
    Compiling .../core/common_runtime/threadpool_device_factory.cc; 4s local
    Compiling .../core/common_runtime/optimization_registry.cc; 4s local
    Compiling tensorflow/core/common_runtime/local_device.cc; 2s local
    Compiling tensorflow/core/graph/quantize_training.cc; 1s local
    Compiling tensorflow/core/comm

[797 / 808] 8 actions running
    Compiling tensorflow/core/graph/quantize_training.cc; 8s local
    Compiling tensorflow/core/common_runtime/executor.cc; 7s local
    Compiling tensorflow/core/framework/dataset.cc; 5s local
    Compiling tensorflow/core/framework/op_segment.cc; 2s local
    Compiling tensorflow/core/graph/optimizer_cse.cc; 2s local
    Compiling tensorflow/core/util/example_proto_helper.cc; 0s local
    Compiling tensorflow/core/common_runtime/memory_types.cc; 0s local
    Compiling tensorflow/core/graph/while_context.cc; 0s local
[797 / 808] 8 actions running
    Compiling tensorflow/core/graph/quantize_training.cc; 8s local
    Compiling tensorflow/core/common_runtime/executor.cc; 7s local
    Compiling tensorflow/core/framework/dataset.cc; 5s local
    Compiling tensorflow/core/framework/op_segment.cc; 3s local
    Compiling tensorflow/core/graph/optimizer_cse.cc; 2s local
    Compiling tensorflow/core/util/example_proto_helper.cc; 0s local
    Compiling tensorflow

[838 / 849] 7 actions running
    Compiling .../core/common_runtime/graph_execution_state.cc; 6s local
    Compiling tensorflow/core/graph/graph_constructor.cc; 4s local
    Compiling tensorflow/core/graph/gradients.cc; 3s local
    Compiling tensorflow/core/common_runtime/session_state.cc; 2s local
    Compiling tensorflow/core/common_runtime/graph_runner.cc; 1s local
    Compiling tensorflow/core/common_runtime/constant_folding.cc; 1s local
    Compiling tensorflow/core/framework/lookup_interface.cc; 0s local
[838 / 849] 8 actions running
    Compiling .../core/common_runtime/graph_execution_state.cc; 6s local
    Compiling tensorflow/core/graph/graph_constructor.cc; 4s local
    Compiling tensorflow/core/graph/gradients.cc; 3s local
    Compiling tensorflow/core/common_runtime/session_state.cc; 2s local
    Compiling tensorflow/core/common_runtime/graph_runner.cc; 1s local
    Compiling tensorflow/core/common_runtime/constant_folding.cc; 1s local
    Compiling tensorflow/core/framew

[850 / 861] 8 actions running
    Compiling tensorflow/core/graph/graph_constructor.cc; 12s local
    Compiling tensorflow/core/common_runtime/constant_folding.cc; 8s local
    Compiling tensorflow/core/common_runtime/renamed_device.cc; 5s local
    Compiling tensorflow/core/common_runtime/copy_tensor.cc; 4s local
    Compiling tensorflow/core/graph/graph.cc; 2s local
    Compiling .../core/common_runtime/parallel_concat_optimizer.cc; 1s local
    Compiling tensorflow/core/common_runtime/function.cc; 0s local
    Compiling tensorflow/core/graph/graph_def_builder_util.cc; 0s local
[853 / 864] 7 actions running
    Compiling tensorflow/core/graph/graph_constructor.cc; 12s local
    Compiling tensorflow/core/common_runtime/constant_folding.cc; 9s local
    Compiling tensorflow/core/common_runtime/copy_tensor.cc; 5s local
    Compiling tensorflow/core/graph/graph.cc; 3s local
    Compiling .../core/common_runtime/parallel_concat_optimizer.cc; 2s local
    Compiling tensorflow/core/common_r

##### Скопируем в Андроид проект

In [57]:
!cp $TFP/bazel-bin/tensorflow/contrib/android/libtensorflow_inference.so $ANDROID/libs/armeabi-v7a

##### Сгенерируем jar с  интерфейсами для работы в Андроид

In [49]:
!cd $TFP && bazel build //tensorflow/contrib/android:android_tensorflow_inference_java

Loading: 
Loading: 0 packages loaded
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (0 packages loaded)
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (0 packages loaded)
    currently loading: tensorflow/java
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (7 packages loaded)
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (7 packages loaded)
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (7 packages loaded)
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (7 packages loaded)
    currently loading: @androidsdk//
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (8 packages loaded)
    currently loading: @bazel_tools//third_party/java/proguard
Analyzing: target //tensorflow/contrib/android:android_tensorflow_inference_ja\
va (9 packages loaded)


INFO: Analysed target //tensorflow/contrib/android:android_tensorflow_inference_java (9 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
[0 / 2] [-----] BazelWorkspaceStatusAction stable-status.txt
[1 / 2] no action
Target //tensorflow/contrib/android:android_tensorflow_inference_java up-to-date:
[4 / 4] no action
  bazel-bin/tensorflow/contrib/android/libandroid_tensorflow_inference_java.jar
[4 / 4] no action
INFO: Elapsed time: 5.952s, Critical Path: 0.01s
[4 / 4] no action
INFO: Build completed successfully, 1 total action


In [52]:
!cp $TFP/bazel-bin/tensorflow/contrib/android/libandroid_tensorflow_inference_java.jar $ANDROID/libs

cp: /Users/rimidalv/AndroidStudioProjects/agronumbers/libs/libandroid_tensorflow_inference_java.jar: Permission denied


In [218]:
!cd $TFP && bazel build tensorflow/contrib/lite/toco:toco

Loading: 
Loading: 0 packages loaded
INFO: Analysed target //tensorflow/contrib/lite/toco:toco (0 packages loaded).
Building: no action
INFO: Found 1 target...
Building: no action
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //tensorflow/contrib/lite/toco:toco up-to-date:
[1 / 1] no action
  bazel-bin/tensorflow/contrib/lite/toco/toco
[1 / 1] no action
INFO: Elapsed time: 1,015s, Critical Path: 0,01s
[1 / 1] no action
INFO: Build completed successfully, 1 total action


In [231]:
!$TFP/bazel-bin/tensorflow/contrib/lite/toco/toco \
--input_file=$PB_MAIN/frozen_opt.pb --output_file=$PB_MAIN/graph_lite.pb \
--input_format=TENSORFLOW_GRAPHDEF --output_format=TENSORFLOW_GRAPHDEF \
--inference_type=FLOAT \
--input_arrays=the_input \
--output_arrays=softmax/truediv \
--input_shapes=1,128,64,1 \
--logtostderr --v=2

2018-01-30 15:46:50.345132: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Enter
2018-01-30 15:46:50.345781: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizeV2
2018-01-30 15:46:50.346457: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Enter
2018-01-30 15:46:50.346493: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizeV2
2018-01-30 15:46:50.347920: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Enter
2018-01-30 15:46:50.347959: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizeV2
2018-01-30 15:46:50.349328: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizeV2
2018-01-30 15:46:50.349361: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting u

2018-01-30 15:46:50.371673: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Requantize
2018-01-30 15:46:50.371697: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizedMatMul
2018-01-30 15:46:50.371712: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: RequantizationRange
2018-01-30 15:46:50.371722: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Requantize
2018-01-30 15:46:50.371734: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: QuantizedMatMul
2018-01-30 15:46:50.371748: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: RequantizationRange
2018-01-30 15:46:50.371758: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1170] Converting unsupported operation: Requantize
2018-01-30 15:46:50.371770: I tensorflow/contrib/lite/t